In [ ]:
import os
from openai import OpenAI
import json
import random
import time

In [ ]:
class APICaller:
    def __init__(self, api_key, endpoint):
        self.client = OpenAI(
            api_key=api_key, 
            base_url=endpoint,
        )

    def call_llm(self, system_prompt, user_prompt, max_tokens=500, temperature=0.5):
        try:
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                max_tokens=max_tokens, 
                temperature=temperature,
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"Ошибка вызова API: {e}")
            return None

In [ ]:
api_key = ""
endpoint = ""
api_caller = APICaller(api_key, endpoint)

In [ ]:
dataset_file = "pirates_dataset_dynamic.jsonl"

dataset = set()
if os.path.exists(dataset_file):
    with open(dataset_file, "r", encoding="utf-8") as f:
        dataset = {json.loads(line)["prompt"] for line in f.readlines()}

topics = [
    "Джек Воробей", "Гектор Барбосса", "Дэйви Джонс", "Уилл Тёрнер", "Элизабет Суонн",
    "Тиа Дальма", "Чёрная Борода", "Летучий Голландец", "Чёрная Жемчужина", "Кодекс пиратов",
    "Проклятие ацтекского золота", "Сундук Дэйви Джонса", "Ост-Индская торговая компания",
    "Лорд Беккет", "Морской Совет пиратов", "Калипсо", "Судьба пиратов после войны",
    "Рождение Дэйви Джонса", "Происхождение Чёрной Бороды", "Пираты Карибского моря и реальная история",
    "Секреты компаса Джека Воробья", "Что случилось с Капитаном Тигом", 
    "Как Барбосса нашёл Чёрную Жемчужину после воскрешения",
    "Отношения Джека Воробья и Энджелики", "Карта, ведущая к Фонтану молодости",
    "Как создавались Пираты Карибского моря", "Какие корабли были у пиратов",
    "Скрытые детали в фильмах", "Какие реальные пираты вдохновили создателей",
    "Как создавались костюмы и реквизит", "Как менялся образ Джека Воробья в фильмах"
]

In [ ]:
system_prompt = "Ты — эксперт по вселенной 'Пираты Карибского моря'. Генерируй новый, уникальный вопрос по вселенной, избегая повторов. Вопросы должны быть детализированными."

num_questions_to_generate = 500 
generated_count = 0

with open(dataset_file, "a", encoding="utf-8") as f: 
    for _ in range(num_questions_to_generate):
        topic = random.choice(topics)
        user_prompt = f"Придумай уникальный, сложный вопрос о {topic} во вселенной 'Пираты Карибского моря'. Избегай общих вопросов."

        question = api_caller.call_llm(system_prompt, user_prompt)
        
        if question and question not in dataset:
            dataset.add(question)

            answer = api_caller.call_llm(system_prompt, f"Ответь кратко и чётко на вопрос: {question}")

            if answer:
                new_entry = {"prompt": question, "response": answer}
                f.write(json.dumps(new_entry, ensure_ascii=False) + "\n") 
                generated_count += 1
                print(f"✅ Добавлен новый вопрос ({generated_count}/{num_questions_to_generate}): {question}")

        if generated_count >= num_questions_to_generate:
            break